In [2]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
# from torchtext.datasets import multi30k, Multi30k
from typing import Iterable, List
from torch.utils.data import DataLoader
from torchmetrics.text import BLEUScore
from transformers.optimization import AdamW
from tqdm.notebook import tqdm_notebook
import torchmetrics
from torch.utils.tensorboard import SummaryWriter

import torch.nn.functional as F
import numpy as np

In [3]:
writer = SummaryWriter('experiments_results')

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/xlm-roberta-large-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Beyons neçə yaşında LaTavia Robertsonla tanış oldu?',
    'context': """Səkkiz yaşında, Beyonce və uşaqlıq dostu Kelly Rowland, bütün qızlardan ibarət əyləncə qrupunun dinləmələrində LaTavia Roberson ilə tanış oldular. Onlar Girl's Tyme kimi daha üç qızla bir qrupa yerləşdirildilər və Hyustondakı istedad şousunda rap etdi və rəqs etdilər. Qrupu gördükdən sonra R&B prodüseri Arne Frager onları Şimali Kaliforniya studiyasına gətirdi və o zamanlar milli televiziyada ən böyük istedad şousu olan Star Search-ə yerləşdirdi. Girl's Tyme qalib gələ bilmədi və Beyonce daha sonra ifa etdikləri mahnının yaxşı olmadığını söylədi. 1995-ci ildə Beyonsun atası qrupu idarə etmək üçün işindən istefa verdi. Bu addım Beyons ailəsinin gəlirini yarıbayarı azaltdı və valideynləri ayrı mənzillərə köçmək məcburiyyətində qaldılar. Mathew orijinal heyəti dördə kəsdi və qrup digər qurulmuş R&B qız qrupları üçün açılış aktı kimi çıxış etməyə davam etdi. Qızlar səsyazma şirkətlərindən əvvəl dinləmələrdən keçdilər və nəhayət, Elektra Records ilə müqavilə imzaladılar, ilk səsyazmaları üzərində işləmək üçün qısa müddətə Atlanta Records-a köçdülər, ancaq şirkət tərəfindən kəsildi. Bu, ailəni daha da gərginləşdirdi və Beyonsun valideynləri ayrıldı. 5 oktyabr 1995-ci ildə Dwayne Wiggins-in Grass Roots Entertainment qrupu ilə müqavilə imzaladı. 1996-cı ildə qızlar Sony Music ilə razılaşma əsasında debüt albomlarını yazmağa başladılar, Knowles ailəsi yenidən birləşdi və qısa müddət sonra qrup Columbia Records ilə müqavilə bağladı.	
"""
}
res = nlp(QA_input)


model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at deepset/xlm-roberta-large-squad2 were not used when initializing XLMRobertaForQuestionAnswering: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForQuestionAnswering from the checkpoint of a model trai

In [5]:
res

{'score': 4.25009748141747e-05,
 'start': 0,
 'end': 15,
 'answer': 'Səkkiz yaşında,'}

In [6]:
model

XLMRobertaForQuestionAnswering(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_feature

In [8]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
# model = AutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

questions = [
    """"19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?"""]

text = r"""Qalan qrup üzvləri 2000-ci ildə çəkilmiş Charlie's Angels filminin soundtrackində görünən "Müstəqil Qadınlar Part I" mahnısını yazdılar. Ardıcıl on bir həftə ərzində ABŞ-ın Billboard Hot 100 cədvəlində birinci yeri tutaraq, onların ən yaxşı chart olan sinqlı oldu. 2001-ci ilin əvvəlində, Destiny's Child üçüncü albomunu tamamlayarkən, Beyonce amerikalı aktyor Mekhi Phifer ilə birlikdə MTV-nin televiziya üçün hazırlanmış "Carmen: A Hip Hopera" filmində böyük rol aldı. Filadelfiyada cərəyan edən film fransız bəstəkarı Georges Bizetin 19-cu əsrdə “Karmen” operasının müasir şərhidir. 2001-ci ilin may ayında üçüncü "Survivor" albomu çıxanda Luckett və Roberson mahnıların onlara yönəldiyini iddia edərək məhkəməyə müraciət etdilər. Albom ilk həftədə 663.000 nüsxə satışı ilə ABŞ Billboard 200-də bir nömrədə debüt etdi. Albom digər bir nömrəli hitləri, "Bootylicious" və baş treki olan "Survivor"u doğurdu, sonuncusu qrupa Duo və ya Vokallı Qrup tərəfindən Ən Yaxşı R&B Performansı üçün Qremmi Mükafatı qazandırdı. 2001-ci ilin oktyabrında 8 Days of Christmas adlı bayram albomunu buraxdıqdan sonra qrup solo karyeralarını davam etdirmək üçün fasilə elan etdi."""



for question in questions:
    inputs = tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt")
    input_ids = inputs["input_ids"].tolist()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    res = model(**inputs)
   
    answer_start = torch.argmax(
        res['start_logits'])
      # Get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(res['end_logits']) + 1  # Get the most likely end of answer with the argmax of the score
    start = F.softmax(res['start_logits'], dim=1)
    end = F.softmax(res['end_logits'], dim=1)

    start_index = torch.argmax((start), dim=1)
    end_index = torch.argmax((end), dim=1)
    
    outer = np.matmul(np.expand_dims(start.detach().numpy(), -1), np.expand_dims(end.detach().numpy(), 1))
    max_answer_len = 512
    candidates = np.tril(np.triu(outer), max_answer_len - 1)
    idx_sorts = [np.argmax(candidates[i].flatten()) for i in range(len(candidates))]
    scores = [candidates[[i], start_index[i], end_index[i]][0] for i in range(len(candidates))]
    
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print(f"Score: {scores}\n")

Question: "19-cu əsrdə "Karmen" operasını hansı fransız bəstəkarı yazmışdır?
Answer: Georges Bizetin

Score: [0.9906479]



In [9]:
from datasets import load_dataset

In [73]:
dataset = load_dataset('vrashad/squad_azerbaijan')

In [79]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",  # truncate context, not the question
        max_length=512,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context.
    # This will help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        # print(examples)
        # print(examples['answer_text'][sample_index])
        answers = {'answer_start':[examples['answer_start'][sample_index]],"text":[examples['answer_text'][sample_index]]}
        # answers = examples["answers"][sample_index]
        # print(answers)
        # If no answers are given, set the cls_index as answer.
        if answers['answer_start'] is not None and answers['text'][0] is not None:
             if len(answers["answer_start"]) == 0:
                 tokenized_examples["start_positions"].append(cls_index)
                 tokenized_examples["end_positions"].append(cls_index)
             else:
                 # Start/end character index of the answer in the text.
                 start_char = answers["answer_start"][0]
                 # print(len(answers["text"]))
                 end_char = start_char + len(answers["text"][0])
     
                 # Start token index of the current span in the text.
                 token_start_index = 0
                 while sequence_ids[token_start_index] != 1:
                     token_start_index += 1
     
                 # End token index of the current span in the text.
                 token_end_index = len(input_ids) - 1
                 while sequence_ids[token_end_index] != 1:
                     token_end_index -= 1
     
                 # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                 if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                     tokenized_examples["start_positions"].append(cls_index)
                     tokenized_examples["end_positions"].append(cls_index)
                 else:
                     # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                     # Note: we could go after the last offset if the answer is the last word (edge case).
                     while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                         token_start_index += 1
                     tokenized_examples["start_positions"].append(token_start_index - 1)
                     while offsets[token_end_index][1] >= end_char:
                         token_end_index -= 1
                     tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [80]:
tokenized_datasets = dataset.map(prepare_train_features, remove_columns=dataset["train"].column_names,batch_size=8)

Map:   0%|          | 0/130319 [00:00<?, ? examples/s]


TypeError: 'float' object is not subscriptable

In [11]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="finetune-BERT-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"].select(range(1000)),
    eval_dataset=tokenized_datasets["validation"].select(range(100)),
    data_collator=data_collator,
    tokenizer=tokenizer,
)